# 🧠 Обнаружение болезни Паркинсона с помощью XGBoost

В этом проекте мы решаем задачу бинарной классификации — предсказание наличия болезни Паркинсона на основе биомедицинских голосовых характеристик.  
Данные взяты из репозитория [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/parkinsons).

## 📥 Загрузка датасета

Скачиваем датасет болезни Паркинсона с официального репозитория UCI:

In [2]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data

--2025-07-26 19:33:31--  https://archive.ics.uci.edu/ml/machine-learning-databases/parkinsons/parkinsons.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘parkinsons.data’

parkinsons.data         [ <=>                ]  39.74K  --.-KB/s    in 0.1s    

2025-07-26 19:33:31 (308 KB/s) - ‘parkinsons.data’ saved [40697]



## 📄 Загрузка и предварительный просмотр данных

Читаем CSV-файл с данными о болезни Паркинсона и выводим первые 5 строк:

In [3]:
import pandas as pd
df = pd.read_csv('parkinsons.data')
df.head()

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


## 🔍 Общая информация о датасете

Выведем форму таблицы и список всех признаков:

In [5]:
print("Форма датасета:", df.shape)
print("\nНазвания признаков:")
print(df.columns.tolist())

Форма датасета: (195, 24)

Названия признаков:
['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']


## 🧹 Предобработка данных

Удалим неинформативный признак `name`, оставим только числовые признаки.  
Целевая переменная — `status` (0 — здоров, 1 — болезнь Паркинсона).

In [6]:
X = df.drop(columns=["name", "status"])
y = df["status"]
print("Признаки:", X.shape)
print("Целевая переменная:", y.shape)

Признаки: (195, 22)
Целевая переменная: (195,)


## ✂️ Разделение данных и масштабирование признаков

Разделим данные на обучающую и тестовую выборки (80/20),  
с учётом сбалансированного распределения целевой переменной (`stratify=y`).  
Затем выполним стандартизацию признаков с помощью `StandardScaler`.

In [38]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## 🚀 Обучение модели XGBoost

Создаём и обучаем классификатор `XGBClassifier` с фиксированным random seed.  
Параметры:
- `use_label_encoder=False` — отключаем устаревший энкодер
- `eval_metric='logloss'` — используем логарифмическую функцию потерь

In [39]:
from xgboost import XGBClassifier

model = XGBClassifier(
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model.fit(X_train_scaled, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/training.py:183: UserWarning: [19:48:25] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

## 📈 Оценка качества модели

Выполним предсказание на тестовой выборке и оценим точность классификации.  
Также выведем подробный отчёт по метрикам (`precision`, `recall`, `f1-score`).

In [40]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

y_pred = model.predict(X_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Точность модели: {accuracy * 100:.2f}%\n")

print("📋 Отчет классификации:\n", classification_report(y_test, y_pred))


🎯 Точность модели: 92.31%

📋 Отчет классификации:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84        10
           1       0.93      0.97      0.95        29

    accuracy                           0.92        39
   macro avg       0.91      0.88      0.90        39
weighted avg       0.92      0.92      0.92        39



## 🏁 Вывод

Модель **XGBoostClassifier** показала хорошую точность на тестовой выборке и продемонстрировала способность различать больных и здоровых с высокой степенью уверенности.

### ✅ Ключевые моменты:

- Использовалась стандартизация признаков (`StandardScaler`)
- Модель обучалась с учётом сбалансированной выборки
- Предсказания оценивались с помощью метрик: `accuracy`, `precision`, `recall`, `f1-score`

### 📊 Полученная точность: **92.31%**

Модель может быть использована как базовая система для медицинского предварительного анализа,  
а также легко расширяется для включения дополнительных признаков или ансамблирования.
